hyperparameter optimization tuning

In [ ]:
!pip install keras-tuner -q

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import random
import keras as ks
import tensorflow as tf
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Flatten, Dense, BatchNormalization, Dropout, GlobalMaxPooling2D, Conv2D
from tensorflow.keras.applications import DenseNet121
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from keras_tuner.tuners import RandomSearch

folders:

In [ ]:
PATH_TRAIN = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/TRAIN'
PATH_TEST = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/TEST'
PATH_VAL = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/VALIDATION'

PATH_TRAIN_CONTROL = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/TRAIN/0'
PATH_TRAIN_ACID = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/TRAIN/1'
PATH_TEST_CONTROL = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/TEST/0'
PATH_TEST_ACID = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/TEST/1'
PATH_VAL_CONTRL = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/VALIDATION/0'
PATH_VAL_ACID = '/content/gdrive/MyDrive/mini-project/bacteria_data/data/data_split_2/VALIDATION/1'

IMG_WIDTH, IMG_HEIGHT = 250, 250
BATCH_SIZE = 16

Generator for train and validation including augmentation

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255,
                                           featurewise_center=False,  # set input mean to 0 over the dataset
                                          samplewise_center=False,  # set each sample mean to 0
                                          featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                          samplewise_std_normalization=False,  # divide each input by its std
                                          zca_whitening=False,  # apply ZCA whitening
                                          rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
                                          width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
                                          height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                                          fill_mode='constant',  # fill shifted pixels with a constant value
                                          cval=0,  # set the constant value to 0 (black)
                                          horizontal_flip=True,  # randomly flip images
                                          vertical_flip=True)  # Generator for our training datavalidation_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our validation data

validation_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our validation data


train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=PATH_TRAIN,
                                                           shuffle=True,
                                                           target_size=(IMG_WIDTH,IMG_HEIGHT),
                                                           class_mode='categorical')
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                             directory=PATH_VAL,
                                                             shuffle=False,
                                                             target_size=(IMG_WIDTH,IMG_HEIGHT),
                                                             class_mode='categorical')

Found 2200 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


optimiztion:

In [ ]:
# Define the model-building function
def build_model(hp):
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    #optional architecture
    inputs = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    x1 = base_model(inputs, training=False)
    x2 = GlobalAveragePooling2D()(x1)
    x3 = Dense(units=hp.Choice("units", [32, 64, 128]), activation="relu")(x2)
    x3 = Dropout(hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1))(x3)
    outputs = Dense(2, activation='softmax')(x3)

    model = tf.keras.Model(inputs, outputs)

    opt = SGD(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]), momentum=0.9)
    model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

    return model

# Define the hyperparameter search space
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # number of different hyperparameter combinations to try
    executions_per_trial=1,  # number of models to train per trial
    directory='/content/gdrive/MyDrive/mini-project/bacteria_data/hyperparameter_results',  # directory to store the search results
    project_name='my_project')  # a name for the search project


In [ ]:
# Define the checkpoint path
checkpoint_path = '/content/gdrive/MyDrive/mini-project/bacteria_data/hyper_results/best_model1.h5'

# Create a ModelCheckpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=False
)

# Perform the hyperparameter search with the ModelCheckpoint callback
tuner.search(
    train_data_gen,
    epochs=5,
    validation_data=val_data_gen,
    callbacks=[checkpoint_callback]
)

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

print(best_model.summary())

Trial 5 Complete [00h 31m 01s]
val_accuracy: 0.9983333349227905

Best val_accuracy So Far: 0.9983333349227905
Total elapsed time: 02h 35m 05s
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, None, None, 1024)  7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               131200    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                   

save the best model:

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save("/content/gdrive/MyDrive/mini-project/bacteria_data/hyperparameter_results/best_model.h5", save_format="h5")

#check if the model saved
best_model_path = '/content/gdrive/MyDrive/mini-project/bacteria_data/hyperparameter_results/best_model.h5'
best_model = load_model(best_model_path)
print(best_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 250, 250, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, None, None, 1024)  7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               131200    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                             